# Project Notebook

## Social Data Science - exam notebook


#### Authors: 
#### Anton Maach-Møller
#### Christian Nøjgaard Fogdal

## 1. Installations

In [138]:
# pip install geopy
# pip install textblob
# pip install vaderSentiment
# pip install emoji
# nltk.download('stopwords')
# nltk.download('vader_lexicon')

## 2. Import of used packages

In [1]:
# import modules
import numpy as np
import pandas as pd
import tweepy as tw #package to handle twitter data
import webbrowser
from geopy.geocoders import Nominatim #Geodata for datacleaning states
from geopy.exc import GeocoderTimedOut #Geodata for datacleaning states
from geopy.extra.rate_limiter import RateLimiter #rate limiter to geopy requests
from tqdm import tqdm #progress bar, use code tqdm.pandas()
from time import sleep #module to control speed of extraction
import re #regex
import random #randomizer
from textblob import TextBlob #To analyse sentiment
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer #Sentiment analyser
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import emoji #to "demojize" - translate emojis to text

import nltk.corpus 
import nltk #Natural Language Toolkit
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer #Stemming and lemmitization
from nltk.stem import WordNetLemmatizer #Stemming and lemmitization

# Machinelearning
from sklearn.model_selection import train_test_split #randomly split dataframe into test and train data
from sklearn.feature_extraction.text import TfidfVectorizer #
from sklearn.linear_model import LogisticRegression

## 3. Extraction of data from Twitter

In [ ]:
OBS
TRY TO USE wait_on_rate_limit 
AND
wait_on_rate_limit_notify

In [127]:

# Unique bearer token - is removed when published, since it is private
MY_BEARER_TOKEN = "AAAAAAAAAAAAAAAAAAAAANcAaQEAAAAA84hzZvsLo8rITqxhQJoQHVAd5wI%3DfD5dXkdwihlc4K08dNi3y84D4lVhjiOIREITen9vVe2UVYxFh7"

# authentication using .client
client = tw.Client(bearer_token=MY_BEARER_TOKEN, wait_on_rate_limit=True)

# query to search for tweets
query = "vaccine lang:en -is:retweet place_country:US has:geo"

# start and end time for fetching tweets
start_time = "2020-11-10T00:00:00Z"
end_time = "2021-01-05T00:00:00Z"

# get tweets from the API
tweets = client.search_all_tweets(   query=query,
                                     start_time=start_time,
                                     end_time=end_time,
                                     tweet_fields = [
                                                     "created_at", 
                                                     "text", 
                                                     "source", 
                                                     #"public_metrics"
                                                    ],
                                  
                                     user_fields = [
                                                    "name", 
                                                    "username", 
                                                    "location", 
                                                    "verified", 
                                                    "description"
                                                   ],
                                  
                                     place_fields=[
                                                   'place_type', 
                                                   'country', 
                                                   'country_code'
                                                  ],
                                  
                                     max_results = 100,
                                     #sort_order = ,
                                     expansions=['author_id', 'geo.place_id']
                                     )

# tweet specific info
print(len(tweets.data))
# user specific info
print(len(tweets.includes["users"]))
print(len(tweets.includes['places']))

# first tweet
first_tweet = tweets.data[0]
dict(first_tweet)

# user information for the first tweet
first_tweet_user = tweets.includes["users"][1]
dict(first_tweet_user)

first_tweet_place = tweets.includes['places'][1]
dict(first_tweet_place)


print("Total Tweets fetched:", len(tweets))

NameError: name 'requests' is not defined

## 4. Creating dataframe from extracted data

In [3]:
# create a list of records
tweet_info_ls = []
# iterate over each tweet and corresponding user details
for tweet, user, place in zip(tweets.data, tweets.includes['users'], tweets.includes['places']):
    tweet_info = {
        'created_at': tweet.created_at,
        'text': tweet.text,
        'source': tweet.source,
        #'public_metrics': tweet.public_metrics,
        #'organic_metrics': tweet.organic_metrics,
        'name': user.name,
        'username': user.username,
        'user_location': user.location,
        'tweet_location': place.full_name,
        'country_code': place.country_code,
        #'country': user.country,
        'verified': user.verified,
        'description': user.description
    }
    tweet_info_ls.append(tweet_info)
# create dataframe from the extracted records
tweets_df = pd.DataFrame(tweet_info_ls)
# display the dataframe
tweets_df['text'][0]

'There’s no reason — NONE. WHATSOEVER. — to waste any dose of the COVID vaccine. \n\nWe’ve ground our way of life to a halt for almost a year. \n\nWe were told a vaccine was the way out of this. \n\nAnd now that we have it...we won’t want as many people as possible to get it?'

## 5. Cleaning location variable and extracting geo data 

### Clean location data before extracting from Nomatim

In [4]:
def cleaner(document_location):
    """
    Cleaning location variable for:
    mentions (@user), 
    removes "#", 
    removes everything after "/" since some users has multiple locations split by "/", - we take the first location.
    try/except is used since the used method cannot be applied on empty values.
    """
    try:
        document_location = re.sub(r'@[^\s]+','',document_location) #removes @user
        document_location = re.sub(r'#[^\s]+','',document_location) #removes #hashtag
        document_location = re.sub(r'\/.*$','',document_location) #removes everything after / since some has multiple "homes", - we take the first
        #document_location = re.sub(r'[\d-]','',document_location) #removes digits
        #document_location = re.sub(r'[^\w\s]','', document_location) #Remove non-alphanumeric characters
        return document_location
    except:
        ("Some variable is None")

tweets_df['user_location'] = tweets_df['user_location'].apply(cleaner)
tweets_df

,created_at,text,source,name,username,user_location,tweet_location,country_code,verified,description
0,2021-01-04 23:59:24+00:00,There’s no reason — NONE. WHATSOEVER. — to was...,Twitter for iPhone,Amy Curtis,RantyAmyCurtis,"Milwaukee, WI","Milwaukee, WI",US,False,"RN, Mom, traditional Catholic, combative nerd...."
1,2021-01-04 23:59:06+00:00,Union Co. will begin to offer Phase 1A Covid-1...,Twitter for iPhone,Committeeman Christopher Capodice,ChrisCapodice,"Springfield, NJ","Springfield, NJ",US,False,"Former Mayor, currently serves as Township Com..."
2,2021-01-04 23:58:24+00:00,@AliNouriPhD @bethnoveck @washingtonpost @US_F...,Twitter for iPhone,Shannon Marino,smarino1002,"Houston, TX","Tomball, TX",US,False,Higher Ed/Student Affairs. Have courage and be...
3,2021-01-04 23:57:27+00:00,if a new wave of infections [as a result of a ...,Twitter for iPhone,Adam Merino,adammerino,New York City,"San Francisco, CA",US,False,@forbes Top Next-Gen Wealth Advisor @SpecOlymp...
4,2021-01-04 23:57:21+00:00,@SpeakerPelosi Yes the Republicans save the am...,Twitter for Android,Sharon Perreault,SharonPerreaul1,None,"Leola, PA",US,False,I am a happily married woman who has 4 childre...
...,...,...,...,...,...,...,...,...,...,...
241,2021-01-04 21:23:49+00:00,@covid_clarity I know our President is making ...,Twitter for iPhone,jet jet,JetDaAirplane,"New Jersey, USA","New Jersey, USA",US,False,"simon: BABY!! :0 me: yes, honey"
242,2021-01-04 21:23:46+00:00,@Amy_Siskind I’m a 70 year-old cancer patient....,Twitter for iPhone,Brandon Shaw 🇺🇸,Bshaw_140,The Gunshine State 🏴‍☠️,"Long Beach, CA",US,False,"'This world is mine, Chico, and everything in ..."
243,2021-01-04 21:21:49+00:00,@nope_its_lily @sajidnizami @rouge_markets @al...,Twitter for Android,Lawrence Smith,LASmithReports,"Louisville, KY",WNYW Fox 5 Studios,US,True,"Communications Director @KentuckyBaptist, @KyT..."
244,2021-01-04 21:21:25+00:00,“Brandenburg told investigators that he believ...,Twitter for iPhone,Jo Langston,TTUprof,None,Oswego Hospital,US,False,"Loving being in my 60s! Christian, mother, fri..."


### Collecting data from Nomatim using GeoPy

In [5]:
geolocator = Nominatim(user_agent="Christian Fogtdal, Student at Copenhagen University, rxw556@alumni.ku.dk")

def city_state_country(row):
    """
    Using Nomatim API to get consistant data of city/state/country from a given country string, 
    - problem was that Twitter users either has "city, state" or "state, country", or something even less usable since location is a free text field.
    
    The code iterates over each row, and extracts city/state/country if applicable.
    Sleep time is set to 1 second. - This is requested by the provider.
    If not applicable the code returns the input.
    """
    coord = f"{row['tweet_location']}"
    sleep(1) #Nomatim is free to use, but requires users to only request 60x/min
    location = geolocator.geocode(coord,exactly_one=True, addressdetails=True)
    if not location:
        # if you see many in a row, it's probably Nominatim blocking you.
        # if it's just every once in a while, there were just some bad results. 
        print('Failed with location: ', coord)
        row['city'], row['state'], row['country'] = None, None, None
        return row
    address = location.raw['address']
    city = address.get('city', '')
    state = address.get('state', '')
    country = address.get('country', '')
    row['city'] = city
    row['state'] = state
    row['country'] = country
    return row

tweets_df = tweets_df.apply(city_state_country, axis=1)
tweets_df

Failed with location:  Macinnes Auditorium @ TGH
Failed with location:  University Of Arkansas For Medical Sciences
Failed with location:  Siebens Building - Mayo Clinic
Failed with location:  Fox News Washington Bureau
Failed with location:  UT Houston Medical School
Failed with location:  Share Our Selves
Failed with location:  WNYW Fox 5 Studios


,created_at,text,source,name,username,user_location,tweet_location,country_code,verified,description,city,state,country
0,2021-01-04 23:59:24+00:00,There’s no reason — NONE. WHATSOEVER. — to was...,Twitter for iPhone,Amy Curtis,RantyAmyCurtis,"Milwaukee, WI","Milwaukee, WI",US,False,"RN, Mom, traditional Catholic, combative nerd....",Milwaukee,Wisconsin,United States
1,2021-01-04 23:59:06+00:00,Union Co. will begin to offer Phase 1A Covid-1...,Twitter for iPhone,Committeeman Christopher Capodice,ChrisCapodice,"Springfield, NJ","Springfield, NJ",US,False,"Former Mayor, currently serves as Township Com...",,New Jersey,United States
2,2021-01-04 23:58:24+00:00,@AliNouriPhD @bethnoveck @washingtonpost @US_F...,Twitter for iPhone,Shannon Marino,smarino1002,"Houston, TX","Tomball, TX",US,False,Higher Ed/Student Affairs. Have courage and be...,,Texas,United States
3,2021-01-04 23:57:27+00:00,if a new wave of infections [as a result of a ...,Twitter for iPhone,Adam Merino,adammerino,New York City,"San Francisco, CA",US,False,@forbes Top Next-Gen Wealth Advisor @SpecOlymp...,San Francisco,California,United States
4,2021-01-04 23:57:21+00:00,@SpeakerPelosi Yes the Republicans save the am...,Twitter for Android,Sharon Perreault,SharonPerreaul1,None,"Leola, PA",US,False,I am a happily married woman who has 4 childre...,,Pennsylvania,United States
...,...,...,...,...,...,...,...,...,...,...,...,...,...
241,2021-01-04 21:23:49+00:00,@covid_clarity I know our President is making ...,Twitter for iPhone,jet jet,JetDaAirplane,"New Jersey, USA","New Jersey, USA",US,False,"simon: BABY!! :0 me: yes, honey",,New Jersey,United States
242,2021-01-04 21:23:46+00:00,@Amy_Siskind I’m a 70 year-old cancer patient....,Twitter for iPhone,Brandon Shaw 🇺🇸,Bshaw_140,The Gunshine State 🏴‍☠️,"Long Beach, CA",US,False,"'This world is mine, Chico, and everything in ...",Long Beach,California,United States
243,2021-01-04 21:21:49+00:00,@nope_its_lily @sajidnizami @rouge_markets @al...,Twitter for Android,Lawrence Smith,LASmithReports,"Louisville, KY",WNYW Fox 5 Studios,US,True,"Communications Director @KentuckyBaptist, @KyT...",None,None,None
244,2021-01-04 21:21:25+00:00,“Brandenburg told investigators that he believ...,Twitter for iPhone,Jo Langston,TTUprof,None,Oswego Hospital,US,False,"Loving being in my 60s! Christian, mother, fri...",City of Oswego,New York,United States


### Final clean of location data

In [7]:
# Removes rows, where country != 'United States'.
# This is often due to free text, where the user for example has written 'my bed' or 'the city'
# I.e. 'the city' returns "City of London", which is why it is removed.
tweets_df = tweets_df.drop(tweets_df[tweets_df.country != 'United States'].index)

#Empty values to NaN
tweets_df['city'].replace('', np.nan, inplace=True)
tweets_df['state'].replace('', np.nan, inplace=True)

# Also removes rows without state info
tweets_df[~tweets_df['state'].isnull()]
tweets_df.dropna(subset=['state'], inplace=True)
len(tweets_df)

239

## 6. Cleaning of the tweet text

In [94]:
## Making a copy, to not be forced to load from the start every time
tweets_df2 = tweets_df.copy()

In [112]:
tweets_df['text'][45]

'If they put the vaccine in beer and opened all the pubs back up, the entire country would be vaccinated by next Thursday.\n\nJust tryna help!'

### Cleaner for the text variable

In [95]:
def cleaner_text(document_tweet):
    """
    Cleaning tweet text for:
    replacing \n newlines with white space ' ',
    lower case everything,
    removes mentions (@user), 
    removes "#" before hashtags and keeps the text, 
    removes digits, 
    removes http-links,
    """
    document_tweet = re.sub(r'\r\n|\r|\n', ' ', document_tweet) #replaces newline with with space
    document_tweet = document_tweet.lower() #To lower case
    document_tweet = re.sub(r'@[^\s]+','',document_tweet) #removes @user
    document_tweet = re.sub(r'[#]','',document_tweet) #Removes only #-sign
    document_tweet = re.sub(r'[\d-]','',document_tweet) #removes digits
    document_tweet = re.sub(r'http[^\s]+','',document_tweet) #removes links starting with https
    document_tweet = re.sub(r'[^\w\s]','', document_tweet) #Remove non-alphanumeric characters
    #document_tweet = re.sub(r'#[^\s]+','',document_tweet) #removes "#" and #hashtagstring
    return document_tweet

tweets_df2['text'] = tweets_df2['text'].apply(cleaner_text)

### Translate emojis to text

In [113]:
def demojize_text(text_tweet):
    """
    translates emojis to text.
    """
    return emoji.demojize(text_tweet, delimiters=("", " "))

tweets_df2['text'] = tweets_df2['text'].apply(demojize_text)

### Removing stop words

In [96]:
def remove_stopwords(tweet_text):
    """
    removes stop words defined by package nltk.corpus
    """
    stop = stopwords.words('english')
    tweet_text = " ".join([word for word in tweet_text.split() if word not in (stop)])
    return tweet_text

tweets_df2['text'] = tweets_df2['text'].apply(remove_stopwords)

### Apply the sentiment analysis using Vader

In [97]:
# Vader sentiment analysis
def sentiment_analyser(tweet_text):
    """
    Using Vader as sentiment analyser.
    Vader outputs neg, neu, pos and compound, - we only keep compound.
    This is done for each tweet.
    """
    analyser= SentimentIntensityAnalyzer()
    sentiment_score = analyser.polarity_scores(tweet_text)["compound"]
    return sentiment_score

# sentiment score from -1 to 1
tweets_df2['sentiment_score'] = tweets_df2['text'].apply(sentiment_analyser)

# boolean value is 0 if sentiment is negative, else 1
tweets_df2['tweet_sentiment'] = [0 if i < 0 else 1 for i in tweets_df2.sentiment_score]

In [111]:
tweets_df2_test = tweets_df2.loc[tweets_df2['state'] == 'Florida']
tweets_df2_test

,created_at,text,source,name,username,user_location,tweet_location,country_code,verified,description,city,state,country,sentiment_score,tweet_sentiment
30,2021-01-04 23:48:28+00:00,big thanks smooth vaccine rollout grateful rec...,Twitter for iPhone,Chris Janson,cajansona,"Jacksonville, FL","Florida, USA",US,False,"Community-focused writer, dad, teacher and cit...",NaN,Florida,United States,0.7096,1
45,2021-01-04 23:39:50+00:00,put vaccine beer opened pubs back entire count...,Twitter for iPhone,Michael McIntyre,McIntyre1937,"Florida, USA","West Samoset, FL",US,False,,NaN,Florida,United States,0.4019,1
48,2021-01-04 23:36:30+00:00,igottheshot pfizerbiontech covid vaccine dose ...,Twitter for iPhone,Candyce McCulloch,candycetaylor4,Ur Mom,"West Palm Beach, FL",US,False,cstat🤩 check out the link below!,West Palm Beach,Florida,United States,0.4939,1
63,2021-01-04 23:29:20+00:00,got first portion vaccine today,Twitter for iPhone,Lola Star,starlola2,"Honolulu, HI",Daytona Beach Municipal Stadium,US,False,liberalism is a mental disorder,Daytona Beach,Florida,United States,0.0000,1
67,2021-01-04 23:27:20+00:00,believe educators one first get vaccine learne...,Twitter for iPhone,Chris Luther,cluther_wmc5,"Memphis, TN","Sunrise, FL",US,True,Social Media Specialist | Former Reporter at W...,Sunrise,Florida,United States,0.0000,1
77,2021-01-04 23:18:33+00:00,anyone type idea regular people get vaccine in...,Twitter for iPhone,Victor Polanco,VictorPolancoo,Sivar- US.,"St Petersburg, FL",US,False,IG : @mrpolanco19,Saint Petersburg,Florida,United States,-0.7624,0
83,2021-01-04 23:15:06+00:00,people need realize reason covid vaccine isnt ...,Twitter for Android,Kevin B Mahoney,kevinbmahoney,Philadelphia PA,"Altamonte Springs, FL",US,False,"Chief Executive Officer, University of Pennsyl...",NaN,Florida,United States,0.0258,1
89,2021-01-04 23:13:10+00:00,thanks presidents warpspeed program vaccine di...,Twitter for iPhone,Tim West,timwestboston,"Boston, MA","Fort Lauderdale, FL",US,False,Lifelong Bostonian who won't shake the accent.,Fort Lauderdale,Florida,United States,0.4404,1
92,2021-01-04 23:11:44+00:00,thought vaccine going nursing home residents r...,Twitter for iPhone,Derrick E Jones,deuceohsixx,"Seattle, WA","Rockledge, FL",US,False,#BlackLivesMatter • #Resistance • #Tech • #Dem...,NaN,Florida,United States,0.5574,1
93,2021-01-04 23:11:28+00:00,like vaccine kill new strain yo man youre pray...,Twitter for iPhone,Alfonso S.,allponcho,"California, USA","Indialantic, FL",US,False,"33, Sagittarius. Love and let be loved.",NaN,Florida,United States,-0.0772,0


In [99]:


df_new = tweets_df2.groupby('state') \
       .agg(count=('state', 'size'), mean_sentiment=('sentiment_score', 'mean')) \
       .reset_index()

df_new2 = df_new.loc[df_new['count'] > 4]

df_new3 = df_new2.sort_values('mean_sentiment', inplace=False, kind='quicksort', na_position='last')
df_new3

,state,count,mean_sentiment
16,Kentucky,7,-0.226643
1,Arizona,5,-0.208520
3,California,27,-0.090719
39,Tennessee,5,-0.064440
40,Texas,29,-0.042231
41,Virginia,9,0.057022
42,Washington,7,0.076971
30,New York,14,0.077136
28,New Jersey,8,0.114163
8,Florida,22,0.166414


In [83]:
tweets_df2

,created_at,text,source,name,username,user_location,tweet_location,country_code,verified,description,city,state,country,sentiment_score,tweet_sentiment
0,2021-01-04 23:59:24+00:00,theres no reason none whatsoever to waste an...,Twitter for iPhone,Amy Curtis,RantyAmyCurtis,"Milwaukee, WI","Milwaukee, WI",US,False,"RN, Mom, traditional Catholic, combative nerd....",Milwaukee,Wisconsin,United States,-0.0232,0
1,2021-01-04 23:59:06+00:00,union co will begin to offer phase a covid vac...,Twitter for iPhone,Committeeman Christopher Capodice,ChrisCapodice,"Springfield, NJ","Springfield, NJ",US,False,"Former Mayor, currently serves as Township Com...",NaN,New Jersey,United States,0.7269,1
2,2021-01-04 23:58:24+00:00,my family is ready and willing but were un...,Twitter for iPhone,Shannon Marino,smarino1002,"Houston, TX","Tomball, TX",US,False,Higher Ed/Student Affairs. Have courage and be...,NaN,Texas,United States,-0.5483,0
3,2021-01-04 23:57:27+00:00,if a new wave of infections as a result of a n...,Twitter for iPhone,Adam Merino,adammerino,New York City,"San Francisco, CA",US,False,@forbes Top Next-Gen Wealth Advisor @SpecOlymp...,San Francisco,California,United States,0.3818,1
4,2021-01-04 23:57:21+00:00,yes the republicans save the america they clo...,Twitter for Android,Sharon Perreault,SharonPerreaul1,None,"Leola, PA",US,False,I am a happily married woman who has 4 childre...,NaN,Pennsylvania,United States,0.7783,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
240,2021-01-04 21:24:21+00:00,if you have ever accidentally drank from your ...,Twitter for iPhone,Steve,iim_riskyy,Rhode Island,"Corpus Christi, TX",US,False,,Corpus Christi,Texas,United States,0.0015,1
241,2021-01-04 21:23:49+00:00,i know our president is making many phone cal...,Twitter for iPhone,jet jet,JetDaAirplane,"New Jersey, USA","New Jersey, USA",US,False,"simon: BABY!! :0 me: yes, honey",NaN,New Jersey,United States,0.4215,1
242,2021-01-04 21:23:46+00:00,im a yearold cancer patient my oncologist to...,Twitter for iPhone,Brandon Shaw 🇺🇸,Bshaw_140,The Gunshine State 🏴‍☠️,"Long Beach, CA",US,False,"'This world is mine, Chico, and everything in ...",Long Beach,California,United States,-0.2144,0
244,2021-01-04 21:21:25+00:00,brandenburg told investigators that he believe...,Twitter for iPhone,Jo Langston,TTUprof,None,Oswego Hospital,US,False,"Loving being in my 60s! Christian, mother, fri...",City of Oswego,New York,United States,-0.8362,0


## Machine learning

### test train split

In [100]:
tweets_df3 = tweets_df2.copy()

In [101]:
# list of tweet sentiments
tweet_sentiments = [0 if i<0 else 1 for i in tweets_df3.sentiment_score]

In [102]:

# splitting dataset into test and train
df_train, df_test = train_test_split(tweets_df3, test_size=.5, random_state=1337)


In [103]:
# Sort the data randomly to mix positive and negative reviews
np.random.seed(0)
df_train = tweets_df2.reindex(np.random.permutation(df_train.index))
df_test = tweets_df2.reindex(np.random.permutation(df_test.index))

# Take out X and Y variable
x_train = df_train['text'].values
x_test = df_test['text'].values
y_train = df_train['tweet_sentiment'].values
y_test = df_test['tweet_sentiment'].values

# Make our bag of words
tfidf = TfidfVectorizer()
x_train_bag = tfidf.fit_transform(x_train)

# Fit the model
lr_text = LogisticRegression(random_state=0) #Text classifier
lr_text.fit(x_train_bag,y_train)

LogisticRegression(random_state=0)

In [104]:
tweets_bag = tfidf.transform(tweets_df3['text'].values)

preds = lr_text.predict(tweets_bag)
print("Testing accuracy =",np.mean([(preds==tweets_df2['tweet_sentiment'])]))

Testing accuracy = 0.6820083682008368


### ML v2

In [90]:
tweets_df3.head()

,created_at,text,source,name,username,user_location,tweet_location,country_code,verified,description,city,state,country,sentiment_score,tweet_sentiment
0,2021-01-04 23:59:24+00:00,theres no reason none whatsoever to waste an...,Twitter for iPhone,Amy Curtis,RantyAmyCurtis,"Milwaukee, WI","Milwaukee, WI",US,False,"RN, Mom, traditional Catholic, combative nerd....",Milwaukee,Wisconsin,United States,-0.0232,0
1,2021-01-04 23:59:06+00:00,union co will begin to offer phase a covid vac...,Twitter for iPhone,Committeeman Christopher Capodice,ChrisCapodice,"Springfield, NJ","Springfield, NJ",US,False,"Former Mayor, currently serves as Township Com...",NaN,New Jersey,United States,0.7269,1
2,2021-01-04 23:58:24+00:00,my family is ready and willing but were un...,Twitter for iPhone,Shannon Marino,smarino1002,"Houston, TX","Tomball, TX",US,False,Higher Ed/Student Affairs. Have courage and be...,NaN,Texas,United States,-0.5483,0
3,2021-01-04 23:57:27+00:00,if a new wave of infections as a result of a n...,Twitter for iPhone,Adam Merino,adammerino,New York City,"San Francisco, CA",US,False,@forbes Top Next-Gen Wealth Advisor @SpecOlymp...,San Francisco,California,United States,0.3818,1
4,2021-01-04 23:57:21+00:00,yes the republicans save the america they clo...,Twitter for Android,Sharon Perreault,SharonPerreaul1,None,"Leola, PA",US,False,I am a happily married woman who has 4 childre...,NaN,Pennsylvania,United States,0.7783,1


In [105]:
# Make our bag of words
tfidf_tweet = TfidfVectorizer()
tweets_new_bag = tfidf_tweet.fit_transform(tweets_df3['text'].values)

# Fit the model
lr_tweets = LogisticRegression(random_state=0) #Text classifier
lr_tweets.fit(tweets_new_bag, tweet_sentiments)


LogisticRegression(random_state=0)

In [107]:

# Make our bag of words
tweets_features = ['_'.join(s.split()) for s in tfidf_tweet.get_feature_names_out()]
tweets_coefficients = lr_tweets.coef_
tweets_coefs_df = pd.DataFrame.from_records(tweets_coefficients, columns=tweets_features)

# Most important (positive and negative) features:
print()
print(tweets_coefs_df.T.sort_values(by=[0], ascending=False).head(50))
print()
print(tweets_coefs_df.T.sort_values(by=[0], ascending=True).head(50))



                     0
got           0.875174
need          0.572002
vaccine       0.511141
today         0.476357
covid         0.472806
make          0.453994
return        0.417495
update        0.404519
one           0.368894
next          0.366469
first         0.361679
rona          0.355477
done          0.334313
received      0.328062
great         0.321149
right         0.311540
get           0.310537
system        0.307344
good          0.299067
peoples       0.297170
shot          0.291972
signed        0.289751
distributed   0.284186
mom           0.283787
metal         0.282932
thanks        0.275665
end           0.272070
available     0.269083
grateful      0.266864
away          0.266731
announced     0.262456
johnson       0.260328
appointments  0.254507
definitely    0.254136
man           0.250894
biden         0.250274
january       0.248600
working       0.247228
enough        0.244003
vaccines      0.242781
sitting       0.236780
please        0.235088
youve     

### Tokenization

In [93]:
text_tokens = nltk.tokenize.word_tokenize(text)
text_tokens

['received',
 'first',
 'two',
 'covid',
 'vaccine',
 'today',
 'first',
 'responder',
 'thanks',
 'personnel',
 'great',
 'job',
 'us']

### Stemming

In [66]:
porter = nltk.PorterStemmer()
review_stemmed = [porter.stem(i) for i in review_nostop]
review_stemmed

received first two covid vaccine today first responder. thanks personnel great job us! = received first two covid vaccine today first responder. thanks personnel great job us!


## 7. Scraping of 2020 US presidential election results from Wiki

In [118]:
df_table = pd.read_html('https://en.wikipedia.org/wiki/2020_United_States_presidential_election')
table_election_state = df_table[19]
table_election_state

# rename state
# new variable state=Red/Blue
# Remove unnecessesary variables

State ordistrict  Biden/HarrisDemocratic                          \
   State ordistrict                   Votes                       %   
0             Alab.                  849624                  36.57%   
1            Alaska                  153778                  42.77%   
2           Arizona                 1672143                  49.36%   
3              Ark.                  423932                  34.78%   
4            Calif.                11110250                  63.48%   
5             Colo.                 1804352                  55.40%   
6             Conn.                 1080831                  59.26%   
7              Del.                  296268                  58.74%   
8              D.C.                  317323                  92.15%   
9           Florida                 5297045                  47.86%   
10          Georgia                 2473633                  49.47%   
11           Hawaii                  366130                  63.73%   
12            Idaho                  287021                  33.07%   
13         Illinois                 3471915                  57.54%   
14          Indiana                 1242416                  40.96%   
15             Iowa                  759061                  44.89%   
16           Kansas                  570323                  41.56%   
17              Ky.                  772474                  36.15%   
18              La.                  856034                  39.85%   
19          Maine †                  435072                  53.09%   
20             ME-1                  266376                  60.11%   
21             ME-2                  168696                  44.82%   
22              Md.                 1985023                  65.36%   
23            Mass.                 2382202                  65.60%   
24            Mich.                 2804040                  50.62%   
25            Minn.                 1717077                  52.40%   
26            Miss.                  539398                  41.06%   
27              Mo.                 1253014                  41.41%   
28            Mont.                  244786                  40.55%   
29           Neb. †                  374583                  39.17%   
30             NE-1                  132261                  41.09%   
31             NE-2                  176468                  51.95%   
32             NE-3                   65854                  22.34%   
33          Nev.[p]                  703486                  50.06%   
34             N.H.                  424937                  52.71%   
35          N.J.[q]                 2608335                  57.33%   
36             N.M.                  501614                  54.29%   
37          N.Y.[r]                 5230985                  60.86%   
38             N.C.                 2684292                  48.59%   
39             N.D.                  114902                  31.76%   
40             Ohio                 2679165                  45.24%   
41            Okla.                  503890                  32.29%   
42           Oregon                 1340383                  56.45%   
43              Pa.                 3458229                  50.01%   
44             R.I.                  307486                  59.39%   
45             S.C.                 1091541                  43.43%   
46             S.D.                  150471                  35.61%   
47            Tenn.                 1143711                  37.45%   
48         Texas[t]                 5259126                  46.48%   
49             Utah                  560282                  37.65%   
50              Vt.                  242820                  66.09%   
51              Va.                 2413568                  54.11%   
52            Wash.                 2369612                  57.97%   
53            W.Va.                  235984                  29.69%   
54             Wis.        

In [96]:
# textblob
res = TextBlob(text_tokens)
print(res.sentiment.polarity)

# Vader
sid_obj= SentimentIntensityAnalyzer()
print(sid_obj.polarity_scores(text_tokens))

TypeError: The `text` argument passed to `__init__(text)` must be a string, not <class 'list'>

In [222]:
# Function used to geocode locations and override timeout error
def do_geocode(address):
    geopy = Nominatim(user_agent="Twitter_data")
    try:
        return geopy.geocode(address,exactly_one=True, addressdetails=True)
    except GeocoderTimedOut:
        return do_geocode(address)

# Creating Geocoded Location column
tweets_df['GeocodedLocation']=tweets_df['location'].apply(lambda x: do_geocode(x) if x != None else None)


0    (Washington, District of Columbia, United Stat...
1    (City of New York, New York, United States, (4...
2    (Texas, United States, (31.2638905, -98.5456116))
3    (Bridgewater, Plymouth County, Massachusetts, ...
4    (Annapolis, Anne Arundel County, Maryland, Uni...
5                                                 None
6    (City of London, Greater London, England, Unit...
7    (Tyler, Smith County, Texas, United States, (3...
Name: GeocodedLocation, dtype: object


In [221]:
# Use the geopy locator with Nominatim/OpenStreetMaps
locator = Nominatim(user_agent='myGeocoder')
adress = ['New York City']
location = locator.geocode(adress,exactly_one=True, addressdetails=True)
location.raw


county = (location.raw['address']).get('state')
county

'New York'

In [207]:

geolocator = Nominatim(user_agent="geoapiExercises")

def city_state_country(adress):
    location = geolocator.geocode(adress, exactly_one=True)
    address = location.raw['address']
    state = address.get('state', '')
    country = address.get('country', '')
    return city, state, country


def do_geocode(address):
    geopy = Nominatim(user_agent="Twitter_data")
    try:
        return geopy.geocode(address,exactly_one=True, addressdetails=True)
    except GeocoderTimedOut:
        return do_geocode(address)

# Creating Geocoded Location column

tweets_df['state']=tweets_df['location'].apply(lambda x: do_geocode(x) if x != None else None)
tweets_df




AttributeError: 'NoneType' object has no attribute 'raw'

In [213]:
from geopy.geocoders import Nominatim
geolocator = Nominatim(user_agent="geoapiExercises")

state1 = "Denmark"
print("State Name:",state1)
location = geolocator.geocode(state1)
print("State Name/Country Name: ")
print(location.address)


State Name: Denmark
State Name/Country Name: 
Danmark


In [192]:
# Function used to geocode locations and override timeout error
def do_geocode(address):
    try:
        geolocator = Nominatim(user_agent="Twitter_location_to_us_state")
        location = geolocator.geocode(adress, addressdetails=True)
        state = location.raw['address']['state']
        return state
        #return geopy.geocode(address,exactly_one=True, addressdetails=True)
    except GeocoderTimedOut:
        return do_geocode(address)

# Creating Geocoded Location column
tweets_df['state']=tweets_df['location'].apply(lambda x: do_geocode(x) if x != None else None)

NameError: name 'adress' is not defined

In [193]:
tweets_df

,created_at,text,source,public_metrics,name,username,location,State_country,country_code,verified,description
0,2020-12-31 23:39:59+00:00,Scientists: We have a #vaccine! \nAMERICANnnnn...,Twitter for Android,None,Captain Co-dependent,TaeX2times,"Washington, DC","Arlington, VA",US,False,🍋🍋🥝🍉🍑 AquaGirl ♒\nHost/ Creator of the Normal...
1,2020-12-31 23:38:46+00:00,The #vaccine rollout is a mess,Twitter for iPhone,None,George Henderson,GeoPHenderson,New York City,"Manhattan, NY",US,False,"Party Planner, Dandy, Man About Town, Former D..."
2,2020-12-31 23:26:10+00:00,"All these ""pro-life"" catholics who refuse to w...",Twitter for Android,None,McRector,lb_rector,Texas,"Texas, USA",US,False,Music lover. Film fanatic. Craft beer enthusia...
3,2020-12-31 22:27:35+00:00,#goodbye2020 #hello2021 #maskup #keephysicaldi...,Instagram,None,"Joe Oravecz, Ph.D.",DrJoeO,"Bridgewater, MA","Bridgewater, MA",US,False,"Vice President: @BridgeStateU ""All About the S..."
4,2020-12-31 21:52:50+00:00,They won't say WHY?! W.T.H.! Do they know some...,Twitter for Android,None,RealMaryZ,RealMaryz,"Annapolis, Maryland","Londontowne, MD",US,False,"The Earth is the Mother of all people, and all..."
5,2020-12-31 20:53:13+00:00,Just in time for the #NewYear!\n\nGrateful for...,Twitter for iPhone,None,"aL'Ai alvarez, MD",alvarezzzy,#FirstGenMD at Stanford,"San Francisco, CA",US,False,"Well-Being Director|Co-Chair, Human Potential ..."
6,2020-12-31 20:46:31+00:00,Look.\n\nEverybody knows that the vaccine is a...,Twitter for Android,None,🇺🇦(((The Dividist))) 🇺🇦,Dividist,The City,"Tyler, TX",US,False,"Social liberal, fiscal conservative, civil lib..."
7,2020-12-31 20:42:16+00:00,Just got my first #shot of the #vaccine!,Twitter for iPhone,None,Paul Buchanan,celadon100,"Tyler, Texas, USA","Chicago, IL",US,False,I am a geochemist who does a bit of research a...


In [102]:
tweets_df.GeocodedLocation[1]

0                                                 None
1    (Las Vegas, Clark County, Nevada, United State...
2    (Plymouth, Hennepin County, Minnesota, United ...
3    (Orlando, Orange County, Florida, United State...
4    (Brooklyn, Kings County, City of New York, New...
5    (City of New York, New York, United States, (4...
6    (Fort Lauderdale, Broward County, Florida, Uni...
7    (Boston, Suffolk County, Massachusetts, United...
8    (Memphis, Shelby County, West Tennessee, Tenne...
Name: GeocodedLocation, dtype: object

In [182]:
geolocator = Nominatim(user_agent="dasdasda")
tlocation = geolocator.geocode("Fort Lauderdale")
print(location.address.state)


AttributeError: 'str' object has no attribute 'state'

In [198]:
from geopy.geocoders import Nominatim

# address is a String e.g. 'Berlin, Germany'
# addressdetails=True does the magic and gives you also the details
location = geolocator.geocode("#FirstGenMD at Stanford", addressdetails=True)

location.raw['address']

AttributeError: 'NoneType' object has no attribute 'raw'

In [143]:
class location

SyntaxError: invalid syntax (952703290.py, line 1)

In [148]:
people = {'adress': {'state': 'Minnesota', 'age': '27', 'sex': 'Male'},
          2: {'name': 'Marie', 'age': '22', 'sex': 'Female'}}

print(people['adress']['state'])
#print(people[1]['age'])
#print(people[1]['sex'])

Minnesota


In [149]:
print(location['address']['state'])



TypeError: tuple indices must be integers or slices, not str

In [86]:
tweets_df.location.address

AttributeError: 'Series' object has no attribute 'address'

In [120]:
tweets_df.GeocodedLocation[1][0]

'Las Vegas, Clark County, Nevada, United States'

In [72]:
# Create the Latitude Column
country=[]
for i in tweets_df['GeocodedLocation']:
    if i== None:
        lat.append(None)
    else:
        lat.append(i.latitude)
tweets_df['Latitude']=lat
tweets_df['Latitude'].astype('float')

# Create the Longitude Column
long=[]
for i in tweets_df['GeocodedLocation']:
    if i== None:
        long.append(None)
    else:
        long.append(i.longitude)
tweets_df['Longitude']=long
tweets_df['Longitude'].astype('float')

0                                                 None
1    (Las Vegas, Clark County, Nevada, United State...
2    (Plymouth, Hennepin County, Minnesota, United ...
3    (Orlando, Orange County, Florida, United State...
4    (Brooklyn, Kings County, City of New York, New...
5    (City of New York, New York, United States, (4...
6    (Fort Lauderdale, Broward County, Florida, Uni...
7    (Boston, Suffolk County, Massachusetts, United...
8    (Memphis, Shelby County, West Tennessee, Tenne...
Name: GeocodedLocation, dtype: object

In [62]:
# Ref: https://geopy.readthedocs.io/en/stable/#usage-with-pandas
geolocator = Nominatim(user_agent="Twitter_data")

#geocode = RateLimiter(geolocator.geocode, min_delay_seconds=3, max_retries=5)
#tweets_df["location"] = tweets_df["location"].progress_apply(geocode, language="en") # Some locations are in hindi, chinese. Language=’en’ returns location in english
#tweets_df["coordinates"] = tweets_df["location"].apply(lambda loc: tuple(loc.point) if loc else None)
tweets_df["state"] = tweets_df["location"].apply(lambda loc: loc[0].split(',')[0] if loc else None)
#tweets_df["country"] = tweets_df["location"].apply(lambda loc: loc[0].split(',')[-1] if loc else None)
tweets_df

,created_at,text,source,public_metrics,location,country,country_code,verified,description,GeocodedLocation,state
0,2022-08-10 22:46:23+00:00,Can we utilize the absorbable micro needle pat...,Twitter for iPhone,None,"Minneapolis/St. Paul, MN",M,US,False,"MPH, CIC. Vaccine and Infectious Disease Geek....",None,Mi
1,2022-08-10 22:34:33+00:00,A combined #influenza and #COVID19 mRNA #vacci...,Twitter for Android,None,"Las Vegas, NV",L,US,False,Future of Healthcare & Life Sciences | Science...,"(Las Vegas, Clark County, Nevada, United State...",La
2,2022-08-10 21:04:18+00:00,@michren03 Twitter blocked me after posting th...,Twitter for Android,None,"Plymouth, MN",P,US,False,Freelance Photographer/Photojournalist \n📷 @ca...,"(Plymouth, Hennepin County, Minnesota, United ...",Pl
3,2022-08-10 19:44:28+00:00,To all the “AUTHENTIC/Fake #DL men please go g...,Twitter for iPhone,None,"Orlando, FL",O,US,False,35 Black Male Anything else feel free to ask!!...,"(Orlando, Orange County, Florida, United State...",Or
4,2022-08-10 15:56:16+00:00,Here are some recommended immunizations for ch...,Twitter for iPhone,None,"Brooklyn, NY",B,US,False,"COPO is a full-service nonprofit in Brooklyn, ...","(Brooklyn, Kings County, City of New York, New...",Br
5,2022-08-10 12:45:49+00:00,"All I saw was “sneaky link, who up, come suck ...",Twitter for iPhone,None,New York City,N,US,True,Reporter @CBSNewYork,"(City of New York, New York, United States, (4...",Ne
6,2022-08-10 11:11:56+00:00,FDA issues emergency use authorization for new...,Twitter for iPhone,None,Fort Lauderdale,F,US,False,Lover of life,"(Fort Lauderdale, Broward County, Florida, Uni...",Fo
7,2022-08-10 00:02:57+00:00,Got this notice that my appointment to get my ...,Instagram,None,"Boston, MA",B,US,False,The Official Twitter Account-𝐃𝐚𝐧 𝐁𝐚𝐫𝐨𝐮𝐜𝐡 𝐋𝐚𝐛 a...,"(Boston, Suffolk County, Massachusetts, United...",Bo
8,2022-08-09 15:12:09+00:00,📣 Fresh from the press 📑 @scilmmunology\n\nChe...,Twitter for iPhone,None,"Memphis, TN",M,US,False,Assistant Professor Department of Community & ...,"(Memphis, Shelby County, West Tennessee, Tenne...",Me


In [108]:
# import the pandas library

# create a list of records
tweet_info_ls2 = []
# iterate over each tweet and corresponding user details
for tweet, user in zip(tweets_df, tweets.includes['places']):
    tweet_info2 = {
        'created_at': tweets_df.created_at,
        'text': tweets_df.text,
        'source': tweets_df.source,
        'name': tweets_df.name,
        'username': tweets_df.username,
        'location': tweets_df.location,
        'country': place.full_name,
        'verified': tweets_df.verified,
        'description': tweets_df.description
    }
    tweet_info_ls2.append(tweet_info2)
# create dataframe from the extracted records
tweets_df2 = pd.DataFrame(tweet_info_ls2)
# display the dataframe
tweets_df2


,created_at,text,source,name,username,location,country,verified,description
0,0 2021-12-08 20:38:34+00:00 1 2021-12-08 2...,0 En dag som i dag vil jeg ønske at jeg boe...,0 Twitter for Android 1 Twitter for Andr...,0 Bjarni Klein Olsen 🇫🇴 1...,0 KleinOlsen 1 evafogdk 2 ...,"0 Føroyar, Streimoy 1 ...","0 Vordingborg, Danmark 1 Vordingborg, Da...",0 False 1 False 2 False 3 False 4 ...,"0 Hjemvendt færing, tilbage på øerne efter ..."
1,0 2021-12-08 20:38:34+00:00 1 2021-12-08 2...,0 En dag som i dag vil jeg ønske at jeg boe...,0 Twitter for Android 1 Twitter for Andr...,0 Bjarni Klein Olsen 🇫🇴 1...,0 KleinOlsen 1 evafogdk 2 ...,"0 Føroyar, Streimoy 1 ...","0 Vordingborg, Danmark 1 Vordingborg, Da...",0 False 1 False 2 False 3 False 4 ...,"0 Hjemvendt færing, tilbage på øerne efter ..."
2,0 2021-12-08 20:38:34+00:00 1 2021-12-08 2...,0 En dag som i dag vil jeg ønske at jeg boe...,0 Twitter for Android 1 Twitter for Andr...,0 Bjarni Klein Olsen 🇫🇴 1...,0 KleinOlsen 1 evafogdk 2 ...,"0 Føroyar, Streimoy 1 ...","0 Vordingborg, Danmark 1 Vordingborg, Da...",0 False 1 False 2 False 3 False 4 ...,"0 Hjemvendt færing, tilbage på øerne efter ..."
3,0 2021-12-08 20:38:34+00:00 1 2021-12-08 2...,0 En dag som i dag vil jeg ønske at jeg boe...,0 Twitter for Android 1 Twitter for Andr...,0 Bjarni Klein Olsen 🇫🇴 1...,0 KleinOlsen 1 evafogdk 2 ...,"0 Føroyar, Streimoy 1 ...","0 Vordingborg, Danmark 1 Vordingborg, Da...",0 False 1 False 2 False 3 False 4 ...,"0 Hjemvendt færing, tilbage på øerne efter ..."
4,0 2021-12-08 20:38:34+00:00 1 2021-12-08 2...,0 En dag som i dag vil jeg ønske at jeg boe...,0 Twitter for Android 1 Twitter for Andr...,0 Bjarni Klein Olsen 🇫🇴 1...,0 KleinOlsen 1 evafogdk 2 ...,"0 Føroyar, Streimoy 1 ...","0 Vordingborg, Danmark 1 Vordingborg, Da...",0 False 1 False 2 False 3 False 4 ...,"0 Hjemvendt færing, tilbage på øerne efter ..."
5,0 2021-12-08 20:38:34+00:00 1 2021-12-08 2...,0 En dag som i dag vil jeg ønske at jeg boe...,0 Twitter for Android 1 Twitter for Andr...,0 Bjarni Klein Olsen 🇫🇴 1...,0 KleinOlsen 1 evafogdk 2 ...,"0 Føroyar, Streimoy 1 ...","0 Vordingborg, Danmark 1 Vordingborg, Da...",0 False 1 False 2 False 3 False 4 ...,"0 Hjemvendt færing, tilbage på øerne efter ..."


In [ ]:
import tweepy

client = tweepy.Client(bearer_token=MY_BEARER_TOKEN)

# Replace with your own search query
query = 'covid -is:retweet place_country:DK'

tweets = client.search_all_tweets(query=query, 
                                  tweet_fields=['context_annotations', 'created_at', 'geo'],
                                  place_fields=['place_type', 'country'], 
                                  expansions='geo.place_id', max_results=10)

# Get list of places from includes object
places = {p["id"]: p for p in tweets.includes['places']}

for tweet in tweets.data:
    print(tweet.id)
    print(tweet.text)
    if places[tweet.geo['place_id']]:
        place = places[tweet.geo['place_id']]
        print(place.full_name)
        
        

1557836753897230336
@JohnH1337 @stinuslindgreen Nej pesticider, PFAS, PFOS osv kan ikke give Covid, men det kan gøre dig mindre modstandsdygtig over for sygdom inkl. C. Tænker ikke at man bliver syg af et glas, men over tid bliver det til meget + blandet med hvor vi ellers udsætters for det og så er vi oppe på store mængder ?
København, Danmark
1557662964148326402
@stinuslindgreen Det meste af mit voksne liv har jeg råbt op om klimaforandring, været glad for min stofpose osv.. Der udledes ekstrem meget kemi ud i vores grundvand og direkte ud i vores haner, kemi der under Covid har lært os, om du spiser/drikker det, ifører dig det eller indånder det, så r👇🏻
København, Danmark
1557090026336538626
@mikemwilson @Lil_Pedro4Lyfe Wait… a new covid variant that is…. Er… undetectable? I’ve had fluish symps for two months on/off, mostly on.
København, Danmark
1557022011007852544
@anders_hviid @NatureComms Is COVID-19 one disease or different diseases related to different strains of the virus?
Vo